# Продолжение статьи о ценах на продукт

Модель, которая позволяет оценить, сколько что-либо стоит, исходя из его описания.

## Обработка данных, часть 2

Сегодня мы расширим наш набор данных, чтобы расширить его охват, и превратим его в отличный набор данных для обучения.  
Обработка данных может показаться менее увлекательной, чем другие задачи, над которыми мы работаем, но это важная часть ответственности инженеров LLM и важное ремесло, которое необходимо отточить, чтобы вы могли создавать свои собственные коммерческие решения на основе высококачественных наборов данных.

Набор данных находится здесь:  
https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023

А папка со всеми наборами данных о товарах находится здесь:  
https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/tree/main/raw/meta_categories

## Важное замечание - прочтите меня сначала, пожалуйста

Мы собираемся создать массивный набор данных из 400 000 наименований, охватывающий несколько типов товаров. На седьмой неделе мы будем использовать эти данные для обучения нашей собственной модели. Это довольно большой набор данных, и в зависимости от выбранного вами графического процессора обучение может занять более 20 часов. Это будет действительно весело, но может обойтись в несколько долларов в вычислительных единицах.

В качестве альтернативы, если вы хотите работать быстро и с минимальными затратами, вы можете работать с меньшим набором данных, ориентированным только на бытовую технику. Вы сможете охватить те же этапы обучения; результаты будут хорошими - не такими хорошими, как при использовании полного набора данных, но все равно потрясающими! Если вы предпочитаете сделать это, я настроил альтернативный блокнот jupyter в этой папке под названием "lite.ipynb", который вы должны использовать вместо этого.

Кроме того, если вы предпочитаете, вы можете ускорить обработку всех этих данных, загрузив файлы pickle, которые мы сохраняем в последней ячейке. Файлы pickle доступны здесь: https://drive.google.com/drive/folders/1f_IZGybvs9o0J5sb3xmtTEQB3BXllzrW

In [ ]:
# imports

import os
import random
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import numpy as np
import pickle

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# More imports after HF login

from loaders import ItemLoader
from items import Item

In [ ]:
%matplotlib inline

## Код загрузки товара

Загляните в loaders.py - там есть несколько полезных кодов, которые облегчат нам жизнь

In [ ]:
# Load in the same dataset as last time

items = ItemLoader("Appliances").load()

In [ ]:
# Look for a familiar item..
print(items[1].prompt)

## Как УВЕЛИЧИТЬ МАСШТАБ

Давайте рассмотрим все наборы данных обо всех товарах, которые вы можете найти в крупном розничном магазине бытовой техники - электротоварах, электронике, офисных принадлежностях и сопутствующих товарах, но не об одежде / косметике / книгах.

In [ ]:
dataset_names = [
    "Automotive",
    "Electronics",
    "Office_Products",
    "Tools_and_Home_Improvement",
    "Cell_Phones_and_Accessories",
    "Toys_and_Games",
    "Appliances",
    "Musical_Instruments",
]

In [ ]:
items = []
for dataset_name in dataset_names:
    loader = ItemLoader(dataset_name)
    items.extend(loader.load())

# Now, time for a coffee break!!
# By the way, I put the biggest datasets first.. it gets faster.

In [ ]:
print(f"A grand total of {len(items):,} items")

In [ ]:
# Plot the distribution of token counts again

tokens = [item.token_count for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="skyblue", bins=range(0, 300, 10))
plt.show()

In [ ]:
# Plot the distribution of prices

prices = [item.price for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.1f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="blueviolet", bins=range(0, 1000, 10))
plt.show()

In [ ]:
category_counts = Counter()
for item in items:
    category_counts[item.category]+=1

categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# Bar chart by category
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="goldenrod")
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

plt.xticks(rotation=30, ha='right')

# Add value labels on top of each bar
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# Display the chart
plt.show()

# Цель

Составьте более сбалансированный набор данных с точки зрения цен. В меньшей степени ориентируйтесь на дешевые товары, средняя стоимость которых превышает 60 долларов. Постарайтесь сбалансировать категории - поменьше автомобильных товаров.

In [ ]:
# Create a dict with a key of each price from $1 to $999
# And in the value, put a list of items with that price (to nearest round number)

slots = defaultdict(list)
for item in items:
    slots[round(item.price)].append(item)

In [ ]:
# Create a dataset called "sample" which tries to more evenly take from the range of prices
# And gives more weight to items from categories other than Automotive
# Set random seed for reproducibility

np.random.seed(42)
random.seed(42)
sample = []
for i in range(1, 1000):
    slot = slots[i]
    if i>=240:
        sample.extend(slot)
    elif len(slot) <= 1200:
        sample.extend(slot)
    else:
        weights = np.array([1 if item.category=='Automotive' else 5 for item in slot])
        weights = weights / np.sum(weights)
        selected_indices = np.random.choice(len(slot), size=1200, replace=False, p=weights)
        selected = [slot[i] for i in selected_indices]
        sample.extend(selected)

print(f"There are {len(sample):,} items in the sample")

In [ ]:
# Plot the distribution of prices in sample

prices = [float(item.price) for item in sample]
plt.figure(figsize=(15, 10))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# OK, we did well in terms of raising the average price and having a smooth-ish population of prices
# Let's see the categories

category_counts = Counter()
for item in sample:
    category_counts[item.category]+=1

categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# Create bar chart
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="lightgreen")

# Customize the chart
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

plt.xticks(rotation=30, ha='right')

# Add value labels on top of each bar
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# Display the chart
plt.show()

In [ ]:
# Automotive still in the lead, but improved somewhat
# For another perspective, let's look at a pie

plt.figure(figsize=(12, 10))
plt.pie(counts, labels=categories, autopct='%1.0f%%', startangle=90)

# Add a circle at the center to create a donut chart (optional)
centre_circle = plt.Circle((0,0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('Categories')

# Equal aspect ratio ensures that pie is drawn as a circle
plt.axis('equal')  

plt.show()

# Набор данных подготовлен куратором!

Мы создали отличный набор данных.

Давайте проведем несколько заключительных проверок

In [ ]:
# How does the price vary with the character count of the prompt?

sizes = [len(item.prompt) for item in sample]
prices = [item.price for item in sample]

# Create the scatter plot
plt.figure(figsize=(15, 8))
plt.scatter(sizes, prices, s=0.2, color="red")

# Add labels and title
plt.xlabel('Size')
plt.ylabel('Price')
plt.title('Is there a simple correlation?')

# Display the plot
plt.show()

In [ ]:
def report(item):
    prompt = item.prompt
    tokens = Item.tokenizer.encode(item.prompt)
    print(prompt)
    print(tokens[-10:])
    print(Item.tokenizer.batch_decode(tokens[-10:]))

In [ ]:
report(sample[398000])

## Наблюдение

Интересная особенность токенизатора Llama заключается в том, что каждое число от 1 до 999 преобразуется в 1 токен, почти так же, как мы видели в gpt-4o. Этого нельзя сказать о quin 2, gamma и phi 3, которые преобразуют отдельные цифры в токены. Это действительно оказалось полезным для нашего проекта, хотя и не является обязательным требованием.

# Наконец-то

Пришло время разделить наши данные на набор данных для обучения, тестирования и валидации.

Обычно для целей тестирования используется 5-10% ваших данных, но на самом деле у нас их гораздо больше, чем нам нужно на данный момент. Мы возьмем 400 000 баллов за обучение и зарезервируем 2000 для тестирования, хотя и не будем использовать их все.


In [ ]:
random.seed(42)
random.shuffle(sample)
train = sample[:400_000]
test = sample[400_000:402_000]
print(f"Divided into a training set of {len(train):,} items and test set of {len(test):,} items")

In [ ]:
print(train[0].prompt)

In [ ]:
print(test[0].test_prompt())

In [ ]:
# Plot the distribution of prices in the first 250 test points

prices = [float(item.price) for item in test[:250]]
plt.figure(figsize=(15, 6))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

# Наконец, загрузите свой новый набор данных

Преобразуйте в подсказки и загрузите в Hugging Face hub.

In [ ]:
train_prompts = [item.prompt for item in train]
train_prices = [item.price for item in train]
test_prompts = [item.test_prompt() for item in test]
test_prices = [item.price for item in test]

In [ ]:
# Create a Dataset from the lists

train_dataset = Dataset.from_dict({"text": train_prompts, "price": train_prices})
test_dataset = Dataset.from_dict({"text": test_prompts, "price": test_prices})
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
# Uncomment these lines if you're ready to push to the hub, and replace my name with your HF username

# HF_USER = "ed-donner"
# DATASET_NAME = f"{HF_USER}/pricer-data"
# dataset.push_to_hub(DATASET_NAME, private=True)

In [ ]:
# One more thing!
# Let's pickle the training and test dataset so we don't have to execute all this code next time!

with open('train.pkl', 'wb') as file:
    pickle.dump(train, file)

with open('test.pkl', 'wb') as file:
    pickle.dump(test, file)

## Todos for you:

- Investigate the dataset more!
- Confirm that the tokenizer tokenizes all 3 digit prices into 1 token